In [2]:
using Pkg
Pkg.add(["JuMP", "MAT", "Downloads", "Distances","HiGHS","Random","Statistics","Printf",])
# Optional: Pkg.add("Ipopt") for IPOPT comparison

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed Hwloc_jll ───────────── v2.12.2+0
   Installed Xorg_libpciaccess_jll ─ v0.18.1+0
   Installed Distances ───────────── v0.10.12
   Installed HiGHS_jll ───────────── v1.11.0+1
   Installed MutableArithmetics ──── v1.6.4
   Installed BenchmarkTools ──────── v1.6.0
   Installed MAT ─────────────────── v0.10.7
   Installed MicrosoftMPI_jll ────── v10.1.4+3
   Installed MPIPreferences ──────── v0.1.11
   Installed CodecBzip2 ──────────── v0.8.5
   Installed HDF5_jll ────────────── v1.14.6+0
   Installed BufferedStreams ─────── v1.2.2
   Installed StructTypes ─────────── v1.11.0
   Installed MathOptInterface ────── v1.43.0
   Installed MPICH_jll ───────────── v4.3.1+0
   Installed OpenMPI_jll ─────────── v5.0.8+0
   Installed libaec_jll ──────────── v1.1.4+0
   Installed JSON3 ───────────────── v1.14.3
   Installed HiGHS ───────────────── v1.19.0
   Installed XML2_jll ────────────── v2.13

In [5]:
using JuMP, HiGHS      # final Problem_900x900(0.75)_1.mat
using LinearAlgebra
using MAT
using Downloads
using Printf
using Random
using Statistics

# Set random seed for reproducibility
Random.seed!(23)

# ================== Configuration ================== #
const MAX_LEVELS = 8
const TOL_UNIQUE = 1e-3
const TOL_CONVERGENCE = 1e-6

# ================== Global Data ================== #
n = 900
Q = zeros(n, n)

# Objective and gradient
f(x) = x' * Q * x
grad_f(x) = 2 * Q * x
x0 = rand(n); x0 ./= sum(x0)

# Linear Minimization Oracle for the simplex
function lmo(g)
    i = argmin(g)
    v = zeros(n)
    v[i] = 1.0
    return v
end

# Check if a point is new
function is_new_point(x, points, tol=TOL_UNIQUE)
    all(norm(x - p) >= tol for p in points)
end

# ================== Adaptive Frank-Wolfe Algorithm ================== #
function conditional_gradient_adjustable_scaling(f, grad_f, lmo, x0; max_iter=1000, epsilon=TOL_CONVERGENCE, delta=1e-10, beta=2, gamma=0.5)
    x_prev = copy(x0)
    x_curr = copy(x0)
    values = [f(x_curr)]
    times = [0.0]
    gaps = Float64[]
    L_ks = Float64[]
    steps = Float64[]
    backtrack_counts = Int[]
    gamma_history = [gamma]
    k = 0
    prev_grad = grad_f(x_prev)
    current_f = f(x_curr)
    recent_backtracks = Int[]

    while k < max_iter
        start = time()
        current_grad = grad_f(x_curr)
        v = lmo(current_grad)
        d = v - x_curr
        normd2 = dot(d, d)
        gap = -dot(current_grad, d)

        if gap <= epsilon
            push!(times, time() - start)
            push!(gaps, gap)
            break
        end

        if k == 0
            d0 = ones(length(x0)) / sqrt(length(x0))
            x_temp = x0 + 1e-3 * d0
            L_k = gamma * (norm(grad_f(x0) - grad_f(x_temp)) / (1e-3 * norm(d0)) + delta)
        else
            grad_diff = norm(current_grad - prev_grad)
            x_diff = norm(x_curr - x_prev)
            L_k = gamma * (grad_diff / x_diff + delta)
        end
        Lknormd2 = L_k * normd2
        t_k = min(gap / Lknormd2, 1.0)
        i = 0
        while true
            x_new = x_curr + t_k * d
            new_f = f(x_new)
            if current_f - new_f >= t_k * gap - (Lknormd2 / 2) * t_k^2
                x_prev = copy(x_curr)
                x_curr = x_new
                push!(backtrack_counts, i)
                push!(recent_backtracks, i)
                break
            else
                L_k *= beta
                Lknormd2 = L_k * normd2
                t_k = min(gap / Lknormd2, 1.0)
                i += 1
            end
        end

        if k % 10 == 0 && k > 0
            total_backtracks = sum(recent_backtracks)
            if total_backtracks == 0
                gamma = max(1e-4, gamma * 0.9)
            elseif total_backtracks > 10
                gamma = min(1.0, gamma * 1.1)
            end
            push!(gamma_history, gamma)
            recent_backtracks = Int[]
        elseif k % 10 == 0
            push!(gamma_history, gamma)
            recent_backtracks = Int[]
        else
            push!(recent_backtracks, i)
        end
        k += 1
        iteration_time = time() - start

        current_f = f(x_curr)
        prev_grad = current_grad
        push!(gaps, gap)
        push!(steps, t_k)
        push!(values, current_f)
        push!(times, iteration_time)
        push!(L_ks, L_k)

        if k % 100 == 0
            # println("k=$k, gap=$(round(gap, digits=4)), t_k=$(round(t_k, digits=4)), L_k=$(round(L_ks[end], digits=6)), Time=$(round(times[end], digits=4)), f=$(round(values[end], digits=6))")
        end
    end
    total_time = sum(times)
    return (x_curr, values, times, gaps, L_ks, backtrack_counts, steps, gamma_history, total_time)
end

# ================== Interior Point via Slack (Paper’s Method) ================== #
function find_interior_point(p_i, sites)
    model = Model(HiGHS.Optimizer)
    set_silent(model)
    @variable(model, x[1:n] >= 0)
    @variable(model, τ)
    @constraint(model, sum(x) == 1)
    for j in 1:length(sites)
        if j == findfirst(s -> s ≈ p_i, sites)
            continue
        end
        p_j = sites[j]
        a = p_j - p_i
        b_val = 0.5 * dot(p_j + p_i, p_j - p_i)
        @constraint(model, dot(a, x) <= b_val + τ)
    end
    @objective(model, Min, τ)
    optimize!(model)
    if termination_status(model) == OPTIMAL
        x_sol = value.(x)
        τ_sol = value(τ)
        @info "Interior point LP: τ = $(round(τ_sol, digits=8)), Status = $(τ_sol < -1e-8 ? "Interior" : "Boundary")"
        return τ_sol < -1e-8 ? (x_sol, "Interior") : (x_sol, "Boundary")
    else
        @warn "Interior point LP failed: Infeasible or unbounded"
        if length(sites) == 1
            @info "Returning barycenter as fallback"
            return ones(n) / n, "Interior"
        end
        return nothing, "Infeasible"
    end
end

# ================== Voronoi Partitioning ================== #
function create_voronoi_partitions(sites)
    base_A = vcat(
        ones(1, n),  # sum(x) ≤ 1
        -ones(1, n),  # sum(x) ≥ 1 → -sum(x) ≤ -1
        Matrix{Float64}(I, n, n)  # x ≥ 0
    )
    base_b = vcat(
        1.0,
        -1.0,
        zeros(n)
    )
    partitions = []
    K = length(sites)
    if K == 1
        push!(partitions, (base_A, base_b))
        return partitions
    end
    for i in 1:K
        A_i = copy(base_A)
        b_i = copy(base_b)
        p_i = sites[i]
        for j in 1:K
            i == j && continue
            p_j = sites[j]
            a = (p_j - p_i)'
            b_val = 0.5 * dot(p_j + p_i, p_j - p_i)
            A_i = vcat(A_i, a)
            b_i = vcat(b_i, b_val)
        end
        push!(partitions, (A_i, b_i))
    end
    return partitions
end

# ================== Main VDCG Algorithm ================== #
function voronoi_conditional_gradient(Q, x0)
    # Modified to return total time and number of points
    x_init, _, cg_times, _, _, _, _, _, init_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, x0)
    total_time = init_time
    sites = [x0, x_init]
    num_points = 1  # Count initial CG call
    archive = [(x_init, f(x_init))]
    best_so_far = f(x_init)
    @printf("Initial solution: f(x) = %.6f\n", f(x_init))
    @info "Distance between x0 and x_init: $(norm(x0 - x_init))"

    for level in 1:MAX_LEVELS
        start_time = time()
        @printf("→ VDCG Level %d: %d sites, Current best f(x) = %.6f\n", level, length(sites), best_so_far)
        partitions = create_voronoi_partitions(sites)
        new_sites = []
        for (i, (A_cell, b_cell)) in enumerate(partitions)
            x_inner, status = find_interior_point(sites[i], sites)
            if x_inner === nothing || status == "Infeasible"
                @info "Skipping cell $i: $status"
                continue
            end
            x_stat, _, cg_times, _, _, _, _, _, cg_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, x_inner)
            total_time += cg_time
            num_points += 1  # Count each CG call
            obj = f(x_stat)
            all_points = [s for (s, _) in archive]
            if is_new_point(x_stat, all_points) && is_new_point(x_stat, new_sites)
                push!(new_sites, x_stat)
                push!(archive, (x_stat, obj))
                @printf("  New point found: f(x) = %.6f\n", obj)
                best_so_far = min(best_so_far, obj)
            else
                min_dist = minimum(norm(x_stat - p) for p in all_points)
                @info "Cell $i: New point not added, obj = $(round(obj, digits=6)), min distance = $(round(min_dist, digits=6)), better = $(obj < best_so_far)"
            end
        end
        elapsed = time() - start_time
        total_time += elapsed
        if isempty(new_sites)
            @printf("→ No new points found. Terminating at level %d.\n", level)
            @printf("  🕒 Level %d runtime: %.4f sec | Best objective: %.6f\n", level, elapsed, best_so_far)
            break
        else
            append!(sites, new_sites)
            @printf("  ✅ Level %d completed in %.4f sec | Best objective so far: %.6f\n", level, elapsed, best_so_far)
        end
    end
    best_idx = argmin([obj for (_, obj) in archive])
    x_best, f_best = archive[best_idx]
    return x_best, f_best, archive, sites, total_time, num_points
end

# ================== Load Problem Data ================== #
function load_Q_matrix()
    mat_file = "Problem_900x900(0.75)_1.mat"
    url = "https://github.com/INFORMSJoC/2024.0577/raw/refs/heads/main/data/StQO%20problems/%23Problem_900x900(0.75)_1.mat"
    if !isfile(mat_file)
        @info "Downloading $mat_file..."
        Downloads.download(url, mat_file)
    end
    @info "Loading Q matrix..."
    mat_data = matread(mat_file)
    global Q = mat_data["Q"]
    @info "Q loaded: size $(size(Q)), cond = $(cond(Q))"
    L = 2 * maximum(abs.(eigvals(Q)))
    @info "Lipschitz constant (L): $L"
end

# ================== Run Experiment ================== #
# ================== Run Experiment ================== #
function main()
    load_Q_matrix()
    # Run VDCG
    @printf("\n🚀 Starting VDCG...\n")
    @time x_vdcg, f_vdcg, archive, sites, vdcg_time, vdcg_points = voronoi_conditional_gradient(Q, x0)
    @printf("\n✅ VDCG Best Objective: %.6f\n", f_vdcg)
    @printf("✅ VDCG Total Time: %.4f sec, Total Points: %d\n", vdcg_time, vdcg_points)
    # Standard CG
    @printf("📊 Running Standard CG...\n")
    x_std, _, _, _, _, _, _, _, std_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, x0)
    f_std = f(x_std)
    @printf("📊 Standard CG Objective: %.6f\n", f_std)
    # Multi-start CG with time
    @printf("🔍 Running Multi-start CG (limited by VDCG time )...\n")
    f_multi = Inf
    total_multi_time = 0.0
    points_evaluated = 0
    seed = 23
    while total_multi_time < vdcg_time
        Random.seed!(seed)
        start = time()
        x, _, _, _, _, _, _, _, cg_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, rand(n) ./ sum(rand(n)))
        total_multi_time += time() - start
        points_evaluated += 1
        f_multi = min(f_multi, f(x))
        @printf("  Seed %d: f(x) = %.6f, Time so far: %.4f sec, Points: %d\n", seed, f(x), total_multi_time, points_evaluated)
        seed += 1
    end
    @printf("🔍 Multi-start CG Best: %.6f (Time: %.4f sec, Points: %d)\n", f_multi, total_multi_time, points_evaluated)
    # Summary Table
    @printf("\n📋 Summary of Results Problem_900x900(0.75)_1.mat\n")
    @printf("┌──────────────────────┬─────────────────┬──────────────┬─────────────────┐\n")
    @printf("│ %-20s │ %-15s │ %-12s │ %-15s │\n", "Method", "Objective Value", "Runtime (sec)", "Points Evaluated")
    @printf("├──────────────────────┼─────────────────┼──────────────┼─────────────────┤\n")
    @printf("│ %-20s │ %-15.6f │ %-12.4f │ %-15d │\n", "VDCG", f_vdcg, vdcg_time, vdcg_points)
    @printf("│ %-20s │ %-15.6f │ %-12.4f │ %-15d │\n", "Standard CG", f_std, std_time, 1)
    @printf("│ %-20s │ %-15.6f │ %-12.4f │ %-15d │\n", "Multi-start CG", f_multi, total_multi_time, points_evaluated)
    @printf("└──────────────────────┴─────────────────┴──────────────┴─────────────────┘\n")
end
main()

[ Info: Downloading Problem_900x900(0.75)_1.mat...
[ Info: Loading Q matrix...
[ Info: Q loaded: size (900, 900), cond = 5058.805240572233



🚀 Starting VDCG...
Initial solution: f(x) = -6.991766


[ Info: Lipschitz constant (L): 2853.8445120220167


→ VDCG Level 1: 2 sites, Current best f(x) = -6.991766


[ Info: Distance between x0 and x_init: 0.39125796878389596


...

  New point found: f(x) = -6.764919


[ Info: Interior point LP: τ = -0.09479923, Status = Interior


  New point found: f(x) = -6.815169
  ✅ Level 8 completed in 196.9985 sec | Best objective so far: -7.408815


[ Info: Interior point LP: τ = -0.00271334, Status = Interior


387.450895 seconds (44.73 M allocations: 285.114 GiB, 4.68% gc time, 1.38% compilation time)

✅ VDCG Best Objective: -7.408815
✅ VDCG Total Time: 591.0145 sec, Total Points: 346
📊 Running Standard CG...
📊 Standard CG Objective: -6.991766
🔍 Running Multi-start CG (limited by VDCG time )...
  Seed 23: f(x) = -6.991446, Time so far: 0.8476 sec, Points: 1
  Seed 24: f(x) = -7.099878, Time so far: 1.6721 sec, Points: 2
  Seed 25: f(x) = -6.865822, Time so far: 2.4822 sec, Points: 3
  Seed 26: f(x) = -7.293781, Time so far: 3.2850 sec, Points: 4
  Seed 27: f(x) = -7.102450, Time so far: 4.0317 sec, Points: 5
  Seed 28: f(x) = -7.294409, Time so far: 4.8171 sec, Points: 6
  Seed 29: f(x) = -6.898155, Time so far: 5.5652 sec, Points: 7
  Seed 30: f(x) = -7.083449, Time so far: 6.4147 sec, Points: 8
  Seed 31: f(x) = -6.900859, Time so far: 7.3824 sec, Points: 9
  Seed 32: f(x) = -6.991324, Time so far: 8.3108 sec, Points: 10
  Seed 33: f(x) = -7.127726, Time so far: 9.1660 sec, Points: 11
  Se

In [6]:
using JuMP, HiGHS       # final Problem_900x900(0.75)_2.mat
using LinearAlgebra
using MAT
using Downloads
using Printf
using Random
using Statistics

# Set random seed for reproducibility
Random.seed!(23)

# ================== Configuration ================== #
const MAX_LEVELS = 8
const TOL_UNIQUE = 1e-3
const TOL_CONVERGENCE = 1e-6

# ================== Global Data ================== #
n = 900
Q = zeros(n, n)

# Objective and gradient
f(x) = x' * Q * x
grad_f(x) = 2 * Q * x
x0 = rand(n); x0 ./= sum(x0)

# Linear Minimization Oracle for the simplex
function lmo(g)
    i = argmin(g)
    v = zeros(n)
    v[i] = 1.0
    return v
end

# Check if a point is new
function is_new_point(x, points, tol=TOL_UNIQUE)
    all(norm(x - p) >= tol for p in points)
end

# ================== Adaptive Frank-Wolfe Algorithm ================== #
function conditional_gradient_adjustable_scaling(f, grad_f, lmo, x0; max_iter=1000, epsilon=TOL_CONVERGENCE, delta=1e-10, beta=2, gamma=0.5)
    x_prev = copy(x0)
    x_curr = copy(x0)
    values = [f(x_curr)]
    times = [0.0]
    gaps = Float64[]
    L_ks = Float64[]
    steps = Float64[]
    backtrack_counts = Int[]
    gamma_history = [gamma]
    k = 0
    prev_grad = grad_f(x_prev)
    current_f = f(x_curr)
    recent_backtracks = Int[]

    while k < max_iter
        start = time()
        current_grad = grad_f(x_curr)
        v = lmo(current_grad)
        d = v - x_curr
        normd2 = dot(d, d)
        gap = -dot(current_grad, d)

        if gap <= epsilon
            push!(times, time() - start)
            push!(gaps, gap)
            break
        end

        if k == 0
            d0 = ones(length(x0)) / sqrt(length(x0))
            x_temp = x0 + 1e-3 * d0
            L_k = gamma * (norm(grad_f(x0) - grad_f(x_temp)) / (1e-3 * norm(d0)) + delta)
        else
            grad_diff = norm(current_grad - prev_grad)
            x_diff = norm(x_curr - x_prev)
            L_k = gamma * (grad_diff / x_diff + delta)
        end
        Lknormd2 = L_k * normd2
        t_k = min(gap / Lknormd2, 1.0)
        i = 0
        while true
            x_new = x_curr + t_k * d
            new_f = f(x_new)
            if current_f - new_f >= t_k * gap - (Lknormd2 / 2) * t_k^2
                x_prev = copy(x_curr)
                x_curr = x_new
                push!(backtrack_counts, i)
                push!(recent_backtracks, i)
                break
            else
                L_k *= beta
                Lknormd2 = L_k * normd2
                t_k = min(gap / Lknormd2, 1.0)
                i += 1
            end
        end

        if k % 10 == 0 && k > 0
            total_backtracks = sum(recent_backtracks)
            if total_backtracks == 0
                gamma = max(1e-4, gamma * 0.9)
            elseif total_backtracks > 10
                gamma = min(1.0, gamma * 1.1)
            end
            push!(gamma_history, gamma)
            recent_backtracks = Int[]
        elseif k % 10 == 0
            push!(gamma_history, gamma)
            recent_backtracks = Int[]
        else
            push!(recent_backtracks, i)
        end
        k += 1
        iteration_time = time() - start

        current_f = f(x_curr)
        prev_grad = current_grad
        push!(gaps, gap)
        push!(steps, t_k)
        push!(values, current_f)
        push!(times, iteration_time)
        push!(L_ks, L_k)

        if k % 100 == 0
            # println("k=$k, gap=$(round(gap, digits=4)), t_k=$(round(t_k, digits=4)), L_k=$(round(L_ks[end], digits=6)), Time=$(round(times[end], digits=4)), f=$(round(values[end], digits=6))")
        end
    end
    total_time = sum(times)
    return (x_curr, values, times, gaps, L_ks, backtrack_counts, steps, gamma_history, total_time)
end

# ================== Interior Point via Slack (Paper’s Method) ================== #
function find_interior_point(p_i, sites)
    model = Model(HiGHS.Optimizer)
    set_silent(model)
    @variable(model, x[1:n] >= 0)
    @variable(model, τ)
    @constraint(model, sum(x) == 1)
    for j in 1:length(sites)
        if j == findfirst(s -> s ≈ p_i, sites)
            continue
        end
        p_j = sites[j]
        a = p_j - p_i
        b_val = 0.5 * dot(p_j + p_i, p_j - p_i)
        @constraint(model, dot(a, x) <= b_val + τ)
    end
    @objective(model, Min, τ)
    optimize!(model)
    if termination_status(model) == OPTIMAL
        x_sol = value.(x)
        τ_sol = value(τ)
        @info "Interior point LP: τ = $(round(τ_sol, digits=8)), Status = $(τ_sol < -1e-8 ? "Interior" : "Boundary")"
        return τ_sol < -1e-8 ? (x_sol, "Interior") : (x_sol, "Boundary")
    else
        @warn "Interior point LP failed: Infeasible or unbounded"
        if length(sites) == 1
            @info "Returning barycenter as fallback"
            return ones(n) / n, "Interior"
        end
        return nothing, "Infeasible"
    end
end

# ================== Voronoi Partitioning ================== #
function create_voronoi_partitions(sites)
    base_A = vcat(
        ones(1, n),  # sum(x) ≤ 1
        -ones(1, n),  # sum(x) ≥ 1 → -sum(x) ≤ -1
        Matrix{Float64}(I, n, n)  # x ≥ 0
    )
    base_b = vcat(
        1.0,
        -1.0,
        zeros(n)
    )
    partitions = []
    K = length(sites)
    if K == 1
        push!(partitions, (base_A, base_b))
        return partitions
    end
    for i in 1:K
        A_i = copy(base_A)
        b_i = copy(base_b)
        p_i = sites[i]
        for j in 1:K
            i == j && continue
            p_j = sites[j]
            a = (p_j - p_i)'
            b_val = 0.5 * dot(p_j + p_i, p_j - p_i)
            A_i = vcat(A_i, a)
            b_i = vcat(b_i, b_val)
        end
        push!(partitions, (A_i, b_i))
    end
    return partitions
end

# ================== Main VDCG Algorithm ================== #
function voronoi_conditional_gradient(Q, x0)
    # Modified to return total time and number of points
    x_init, _, cg_times, _, _, _, _, _, init_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, x0)
    total_time = init_time
    sites = [x0, x_init]
    num_points = 1  # Count initial CG call
    archive = [(x_init, f(x_init))]
    best_so_far = f(x_init)
    @printf("Initial solution: f(x) = %.6f\n", f(x_init))
    @info "Distance between x0 and x_init: $(norm(x0 - x_init))"

    for level in 1:MAX_LEVELS
        start_time = time()
        @printf("→ VDCG Level %d: %d sites, Current best f(x) = %.6f\n", level, length(sites), best_so_far)
        partitions = create_voronoi_partitions(sites)
        new_sites = []
        for (i, (A_cell, b_cell)) in enumerate(partitions)
            x_inner, status = find_interior_point(sites[i], sites)
            if x_inner === nothing || status == "Infeasible"
                @info "Skipping cell $i: $status"
                continue
            end
            x_stat, _, cg_times, _, _, _, _, _, cg_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, x_inner)
            total_time += cg_time
            num_points += 1  # Count each CG call
            obj = f(x_stat)
            all_points = [s for (s, _) in archive]
            if is_new_point(x_stat, all_points) && is_new_point(x_stat, new_sites)
                push!(new_sites, x_stat)
                push!(archive, (x_stat, obj))
                @printf("  New point found: f(x) = %.6f\n", obj)
                best_so_far = min(best_so_far, obj)
            else
                min_dist = minimum(norm(x_stat - p) for p in all_points)
                @info "Cell $i: New point not added, obj = $(round(obj, digits=6)), min distance = $(round(min_dist, digits=6)), better = $(obj < best_so_far)"
            end
        end
        elapsed = time() - start_time
        total_time += elapsed
        if isempty(new_sites)
            @printf("→ No new points found. Terminating at level %d.\n", level)
            @printf("  🕒 Level %d runtime: %.4f sec | Best objective: %.6f\n", level, elapsed, best_so_far)
            break
        else
            append!(sites, new_sites)
            @printf("  ✅ Level %d completed in %.4f sec | Best objective so far: %.6f\n", level, elapsed, best_so_far)
        end
    end
    best_idx = argmin([obj for (_, obj) in archive])
    x_best, f_best = archive[best_idx]
    return x_best, f_best, archive, sites, total_time, num_points
end

# ================== Load Problem Data ================== #
function load_Q_matrix()
    mat_file = "Problem_900x900(0.75)_2.mat"
    url = "https://github.com/INFORMSJoC/2024.0577/raw/refs/heads/main/data/StQO%20problems/%23Problem_900x900(0.75)_2.mat"
    if !isfile(mat_file)
        @info "Downloading $mat_file..."
        Downloads.download(url, mat_file)
    end
    @info "Loading Q matrix..."
    mat_data = matread(mat_file)
    global Q = mat_data["Q"]
    @info "Q loaded: size $(size(Q)), cond = $(cond(Q))"
    L = 2 * maximum(abs.(eigvals(Q)))
    @info "Lipschitz constant (L): $L"
end

# ================== Run Experiment ================== #
# ================== Run Experiment ================== #
function main()
    load_Q_matrix()
    # Run VDCG
    @printf("\n🚀 Starting VDCG...\n")
    @time x_vdcg, f_vdcg, archive, sites, vdcg_time, vdcg_points = voronoi_conditional_gradient(Q, x0)
    @printf("\n✅ VDCG Best Objective: %.6f\n", f_vdcg)
    @printf("✅ VDCG Total Time: %.4f sec, Total Points: %d\n", vdcg_time, vdcg_points)
    # Standard CG
    @printf("📊 Running Standard CG...\n")
    x_std, _, _, _, _, _, _, _, std_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, x0)
    f_std = f(x_std)
    @printf("📊 Standard CG Objective: %.6f\n", f_std)
    # Multi-start CG with time
    @printf("🔍 Running Multi-start CG (limited by VDCG time )...\n")
    f_multi = Inf
    total_multi_time = 0.0
    points_evaluated = 0
    seed = 23
    while total_multi_time < vdcg_time
        Random.seed!(seed)
        start = time()
        x, _, _, _, _, _, _, _, cg_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, rand(n) ./ sum(rand(n)))
        total_multi_time += time() - start
        points_evaluated += 1
        f_multi = min(f_multi, f(x))
        @printf("  Seed %d: f(x) = %.6f, Time so far: %.4f sec, Points: %d\n", seed, f(x), total_multi_time, points_evaluated)
        seed += 1
    end
    @printf("🔍 Multi-start CG Best: %.6f (Time: %.4f sec, Points: %d)\n", f_multi, total_multi_time, points_evaluated)
    # Summary Table
    @printf("\n📋 Summary of Results Problem_900x900(0.75)_2.mat\n")
    @printf("┌──────────────────────┬─────────────────┬──────────────┬─────────────────┐\n")
    @printf("│ %-20s │ %-15s │ %-12s │ %-15s │\n", "Method", "Objective Value", "Runtime (sec)", "Points Evaluated")
    @printf("├──────────────────────┼─────────────────┼──────────────┼─────────────────┤\n")
    @printf("│ %-20s │ %-15.6f │ %-12.4f │ %-15d │\n", "VDCG", f_vdcg, vdcg_time, vdcg_points)
    @printf("│ %-20s │ %-15.6f │ %-12.4f │ %-15d │\n", "Standard CG", f_std, std_time, 1)
    @printf("│ %-20s │ %-15.6f │ %-12.4f │ %-15d │\n", "Multi-start CG", f_multi, total_multi_time, points_evaluated)
    @printf("└──────────────────────┴─────────────────┴──────────────┴─────────────────┘\n")
end
main()

[ Info: Downloading Problem_900x900(0.75)_2.mat...
[ Info: Loading Q matrix...
[ Info: Q loaded: size (900, 900), cond = 4807.088566035061



🚀 Starting VDCG...
Initial solution: f(x) = -7.013470


[ Info: Lipschitz constant (L): 2794.858915311439


→ VDCG Level 1: 2 sites, Current best f(x) = -7.013470


[ Info: Distance between x0 and x_init: 0.4123049354898141
[ Info: Interior point LP: τ = -0.08673402, Status = Interior


...

  New point found: f(x) = -6.919389


[ Info: Interior point LP: τ = -0.13146067, Status = Interior


  New point found: f(x) = -6.932061
  ✅ Level 8 completed in 240.0591 sec | Best objective so far: -7.351734


[ Info: Interior point LP: τ = -0.12483114, Status = Interior


463.714756 seconds (49.87 M allocations: 405.661 GiB, 4.75% gc time, 0.19% compilation time)

✅ VDCG Best Objective: -7.351734
✅ VDCG Total Time: 700.2640 sec, Total Points: 417
📊 Running Standard CG...
📊 Standard CG Objective: -7.013470
🔍 Running Multi-start CG (limited by VDCG time )...
  Seed 23: f(x) = -7.100843, Time so far: 0.9404 sec, Points: 1
  Seed 24: f(x) = -7.220287, Time so far: 1.9716 sec, Points: 2
  Seed 25: f(x) = -6.778499, Time so far: 2.9904 sec, Points: 3
  Seed 26: f(x) = -6.771642, Time so far: 4.0406 sec, Points: 4
  Seed 27: f(x) = -7.100726, Time so far: 4.9645 sec, Points: 5
  Seed 28: f(x) = -6.895121, Time so far: 5.8803 sec, Points: 6
  Seed 29: f(x) = -6.958094, Time so far: 6.6690 sec, Points: 7
  Seed 30: f(x) = -6.981919, Time so far: 7.5748 sec, Points: 8
  Seed 31: f(x) = -7.022841, Time so far: 8.4042 sec, Points: 9
  Seed 32: f(x) = -7.135993, Time so far: 9.2877 sec, Points: 10
  Seed 33: f(x) = -7.109673, Time so far: 10.1865 sec, Points: 11
  S

In [7]:
using JuMP, HiGHS       # final Problem_900x900(0.75)_3.mat
using LinearAlgebra
using MAT
using Downloads
using Printf
using Random
using Statistics

# Set random seed for reproducibility
Random.seed!(23)

# ================== Configuration ================== #
const MAX_LEVELS = 8
const TOL_UNIQUE = 1e-3
const TOL_CONVERGENCE = 1e-6

# ================== Global Data ================== #
n = 900
Q = zeros(n, n)

# Objective and gradient
f(x) = x' * Q * x
grad_f(x) = 2 * Q * x
x0 = rand(n); x0 ./= sum(x0)

# Linear Minimization Oracle for the simplex
function lmo(g)
    i = argmin(g)
    v = zeros(n)
    v[i] = 1.0
    return v
end

# Check if a point is new
function is_new_point(x, points, tol=TOL_UNIQUE)
    all(norm(x - p) >= tol for p in points)
end

# ================== Adaptive Frank-Wolfe Algorithm ================== #
function conditional_gradient_adjustable_scaling(f, grad_f, lmo, x0; max_iter=1000, epsilon=TOL_CONVERGENCE, delta=1e-10, beta=2, gamma=0.5)
    x_prev = copy(x0)
    x_curr = copy(x0)
    values = [f(x_curr)]
    times = [0.0]
    gaps = Float64[]
    L_ks = Float64[]
    steps = Float64[]
    backtrack_counts = Int[]
    gamma_history = [gamma]
    k = 0
    prev_grad = grad_f(x_prev)
    current_f = f(x_curr)
    recent_backtracks = Int[]

    while k < max_iter
        start = time()
        current_grad = grad_f(x_curr)
        v = lmo(current_grad)
        d = v - x_curr
        normd2 = dot(d, d)
        gap = -dot(current_grad, d)

        if gap <= epsilon
            push!(times, time() - start)
            push!(gaps, gap)
            break
        end

        if k == 0
            d0 = ones(length(x0)) / sqrt(length(x0))
            x_temp = x0 + 1e-3 * d0
            L_k = gamma * (norm(grad_f(x0) - grad_f(x_temp)) / (1e-3 * norm(d0)) + delta)
        else
            grad_diff = norm(current_grad - prev_grad)
            x_diff = norm(x_curr - x_prev)
            L_k = gamma * (grad_diff / x_diff + delta)
        end
        Lknormd2 = L_k * normd2
        t_k = min(gap / Lknormd2, 1.0)
        i = 0
        while true
            x_new = x_curr + t_k * d
            new_f = f(x_new)
            if current_f - new_f >= t_k * gap - (Lknormd2 / 2) * t_k^2
                x_prev = copy(x_curr)
                x_curr = x_new
                push!(backtrack_counts, i)
                push!(recent_backtracks, i)
                break
            else
                L_k *= beta
                Lknormd2 = L_k * normd2
                t_k = min(gap / Lknormd2, 1.0)
                i += 1
            end
        end

        if k % 10 == 0 && k > 0
            total_backtracks = sum(recent_backtracks)
            if total_backtracks == 0
                gamma = max(1e-4, gamma * 0.9)
            elseif total_backtracks > 10
                gamma = min(1.0, gamma * 1.1)
            end
            push!(gamma_history, gamma)
            recent_backtracks = Int[]
        elseif k % 10 == 0
            push!(gamma_history, gamma)
            recent_backtracks = Int[]
        else
            push!(recent_backtracks, i)
        end
        k += 1
        iteration_time = time() - start

        current_f = f(x_curr)
        prev_grad = current_grad
        push!(gaps, gap)
        push!(steps, t_k)
        push!(values, current_f)
        push!(times, iteration_time)
        push!(L_ks, L_k)

        if k % 100 == 0
            # println("k=$k, gap=$(round(gap, digits=4)), t_k=$(round(t_k, digits=4)), L_k=$(round(L_ks[end], digits=6)), Time=$(round(times[end], digits=4)), f=$(round(values[end], digits=6))")
        end
    end
    total_time = sum(times)
    return (x_curr, values, times, gaps, L_ks, backtrack_counts, steps, gamma_history, total_time)
end

# ================== Interior Point via Slack (Paper’s Method) ================== #
function find_interior_point(p_i, sites)
    model = Model(HiGHS.Optimizer)
    set_silent(model)
    @variable(model, x[1:n] >= 0)
    @variable(model, τ)
    @constraint(model, sum(x) == 1)
    for j in 1:length(sites)
        if j == findfirst(s -> s ≈ p_i, sites)
            continue
        end
        p_j = sites[j]
        a = p_j - p_i
        b_val = 0.5 * dot(p_j + p_i, p_j - p_i)
        @constraint(model, dot(a, x) <= b_val + τ)
    end
    @objective(model, Min, τ)
    optimize!(model)
    if termination_status(model) == OPTIMAL
        x_sol = value.(x)
        τ_sol = value(τ)
        @info "Interior point LP: τ = $(round(τ_sol, digits=8)), Status = $(τ_sol < -1e-8 ? "Interior" : "Boundary")"
        return τ_sol < -1e-8 ? (x_sol, "Interior") : (x_sol, "Boundary")
    else
        @warn "Interior point LP failed: Infeasible or unbounded"
        if length(sites) == 1
            @info "Returning barycenter as fallback"
            return ones(n) / n, "Interior"
        end
        return nothing, "Infeasible"
    end
end

# ================== Voronoi Partitioning ================== #
function create_voronoi_partitions(sites)
    base_A = vcat(
        ones(1, n),  # sum(x) ≤ 1
        -ones(1, n),  # sum(x) ≥ 1 → -sum(x) ≤ -1
        Matrix{Float64}(I, n, n)  # x ≥ 0
    )
    base_b = vcat(
        1.0,
        -1.0,
        zeros(n)
    )
    partitions = []
    K = length(sites)
    if K == 1
        push!(partitions, (base_A, base_b))
        return partitions
    end
    for i in 1:K
        A_i = copy(base_A)
        b_i = copy(base_b)
        p_i = sites[i]
        for j in 1:K
            i == j && continue
            p_j = sites[j]
            a = (p_j - p_i)'
            b_val = 0.5 * dot(p_j + p_i, p_j - p_i)
            A_i = vcat(A_i, a)
            b_i = vcat(b_i, b_val)
        end
        push!(partitions, (A_i, b_i))
    end
    return partitions
end

# ================== Main VDCG Algorithm ================== #
function voronoi_conditional_gradient(Q, x0)
    # Modified to return total time and number of points
    x_init, _, cg_times, _, _, _, _, _, init_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, x0)
    total_time = init_time
    sites = [x0, x_init]
    num_points = 1  # Count initial CG call
    archive = [(x_init, f(x_init))]
    best_so_far = f(x_init)
    @printf("Initial solution: f(x) = %.6f\n", f(x_init))
    @info "Distance between x0 and x_init: $(norm(x0 - x_init))"

    for level in 1:MAX_LEVELS
        start_time = time()
        @printf("→ VDCG Level %d: %d sites, Current best f(x) = %.6f\n", level, length(sites), best_so_far)
        partitions = create_voronoi_partitions(sites)
        new_sites = []
        for (i, (A_cell, b_cell)) in enumerate(partitions)
            x_inner, status = find_interior_point(sites[i], sites)
            if x_inner === nothing || status == "Infeasible"
                @info "Skipping cell $i: $status"
                continue
            end
            x_stat, _, cg_times, _, _, _, _, _, cg_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, x_inner)
            total_time += cg_time
            num_points += 1  # Count each CG call
            obj = f(x_stat)
            all_points = [s for (s, _) in archive]
            if is_new_point(x_stat, all_points) && is_new_point(x_stat, new_sites)
                push!(new_sites, x_stat)
                push!(archive, (x_stat, obj))
                @printf("  New point found: f(x) = %.6f\n", obj)
                best_so_far = min(best_so_far, obj)
            else
                min_dist = minimum(norm(x_stat - p) for p in all_points)
                @info "Cell $i: New point not added, obj = $(round(obj, digits=6)), min distance = $(round(min_dist, digits=6)), better = $(obj < best_so_far)"
            end
        end
        elapsed = time() - start_time
        total_time += elapsed
        if isempty(new_sites)
            @printf("→ No new points found. Terminating at level %d.\n", level)
            @printf("  🕒 Level %d runtime: %.4f sec | Best objective: %.6f\n", level, elapsed, best_so_far)
            break
        else
            append!(sites, new_sites)
            @printf("  ✅ Level %d completed in %.4f sec | Best objective so far: %.6f\n", level, elapsed, best_so_far)
        end
    end
    best_idx = argmin([obj for (_, obj) in archive])
    x_best, f_best = archive[best_idx]
    return x_best, f_best, archive, sites, total_time, num_points
end

# ================== Load Problem Data ================== #
function load_Q_matrix()
    mat_file = "Problem_900x900(0.75)_3.mat"
    url = "https://github.com/INFORMSJoC/2024.0577/raw/refs/heads/main/data/StQO%20problems/%23Problem_900x900(0.75)_3.mat"
    if !isfile(mat_file)
        @info "Downloading $mat_file..."
        Downloads.download(url, mat_file)
    end
    @info "Loading Q matrix..."
    mat_data = matread(mat_file)
    global Q = mat_data["Q"]
    @info "Q loaded: size $(size(Q)), cond = $(cond(Q))"
    L = 2 * maximum(abs.(eigvals(Q)))
    @info "Lipschitz constant (L): $L"
end

# ================== Run Experiment ================== #
# ================== Run Experiment ================== #
function main()
    load_Q_matrix()
    # Run VDCG
    @printf("\n🚀 Starting VDCG...\n")
    @time x_vdcg, f_vdcg, archive, sites, vdcg_time, vdcg_points = voronoi_conditional_gradient(Q, x0)
    @printf("\n✅ VDCG Best Objective: %.6f\n", f_vdcg)
    @printf("✅ VDCG Total Time: %.4f sec, Total Points: %d\n", vdcg_time, vdcg_points)
    # Standard CG
    @printf("📊 Running Standard CG...\n")
    x_std, _, _, _, _, _, _, _, std_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, x0)
    f_std = f(x_std)
    @printf("📊 Standard CG Objective: %.6f\n", f_std)
    # Multi-start CG with time
    @printf("🔍 Running Multi-start CG (limited by VDCG time )...\n")
    f_multi = Inf
    total_multi_time = 0.0
    points_evaluated = 0
    seed = 23
    while total_multi_time < vdcg_time
        Random.seed!(seed)
        start = time()
        x, _, _, _, _, _, _, _, cg_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, rand(n) ./ sum(rand(n)))
        total_multi_time += time() - start
        points_evaluated += 1
        f_multi = min(f_multi, f(x))
        @printf("  Seed %d: f(x) = %.6f, Time so far: %.4f sec, Points: %d\n", seed, f(x), total_multi_time, points_evaluated)
        seed += 1
    end
    @printf("🔍 Multi-start CG Best: %.6f (Time: %.4f sec, Points: %d)\n", f_multi, total_multi_time, points_evaluated)
    # Summary Table
    @printf("\n📋 Summary of Results Problem_900x900(0.75)_3.mat\n")
    @printf("┌──────────────────────┬─────────────────┬──────────────┬─────────────────┐\n")
    @printf("│ %-20s │ %-15s │ %-12s │ %-15s │\n", "Method", "Objective Value", "Runtime (sec)", "Points Evaluated")
    @printf("├──────────────────────┼─────────────────┼──────────────┼─────────────────┤\n")
    @printf("│ %-20s │ %-15.6f │ %-12.4f │ %-15d │\n", "VDCG", f_vdcg, vdcg_time, vdcg_points)
    @printf("│ %-20s │ %-15.6f │ %-12.4f │ %-15d │\n", "Standard CG", f_std, std_time, 1)
    @printf("│ %-20s │ %-15.6f │ %-12.4f │ %-15d │\n", "Multi-start CG", f_multi, total_multi_time, points_evaluated)
    @printf("└──────────────────────┴─────────────────┴──────────────┴─────────────────┘\n")
end
main()

[ Info: Downloading Problem_900x900(0.75)_3.mat...
[ Info: Loading Q matrix...
[ Info: Q loaded: size (900, 900), cond = 35496.75479381207



🚀 Starting VDCG...
Initial solution: f(x) = -6.751235


[ Info: Lipschitz constant (L): 2816.6797504790884


→ VDCG Level 1: 2 sites, Current best f(x) = -6.751235


[ Info: Distance between x0 and x_init: 0.40725347858478633
[ Info: Interior point LP: τ = -0.08487523, Status = Interior


...

  New point found: f(x) = -7.056641


[ Info: Interior point LP: τ = -0.13189306, Status = Interior


  New point found: f(x) = -6.875462
  ✅ Level 8 completed in 276.8890 sec | Best objective so far: -7.360262


[ Info: Interior point LP: τ = -0.00335205, Status = Interior


518.183773 seconds (54.52 M allocations: 448.429 GiB, 4.82% gc time, 0.17% compilation time)

✅ VDCG Best Objective: -7.360262
✅ VDCG Total Time: 779.7218 sec, Total Points: 432
📊 Running Standard CG...
📊 Standard CG Objective: -6.751235
🔍 Running Multi-start CG (limited by VDCG time )...
  Seed 23: f(x) = -6.751671, Time so far: 0.9551 sec, Points: 1
  Seed 24: f(x) = -7.120526, Time so far: 1.8976 sec, Points: 2
  Seed 25: f(x) = -7.148162, Time so far: 2.7784 sec, Points: 3
  Seed 26: f(x) = -6.988015, Time so far: 3.6411 sec, Points: 4
  Seed 27: f(x) = -7.148160, Time so far: 4.6707 sec, Points: 5
  Seed 28: f(x) = -6.844133, Time so far: 5.6670 sec, Points: 6
  Seed 29: f(x) = -7.120479, Time so far: 6.7679 sec, Points: 7
  Seed 30: f(x) = -6.776538, Time so far: 7.7084 sec, Points: 8
  Seed 31: f(x) = -7.148179, Time so far: 8.6004 sec, Points: 9
  Seed 32: f(x) = -7.120349, Time so far: 9.5375 sec, Points: 10
  Seed 33: f(x) = -6.773713, Time so far: 10.4106 sec, Points: 11
  S

In [8]:
using JuMP, HiGHS       # final Problem_900x900(0.75)_4.mat
using LinearAlgebra
using MAT
using Downloads
using Printf
using Random
using Statistics

# Set random seed for reproducibility
Random.seed!(23)

# ================== Configuration ================== #
const MAX_LEVELS = 8
const TOL_UNIQUE = 1e-3
const TOL_CONVERGENCE = 1e-6

# ================== Global Data ================== #
n = 900
Q = zeros(n, n)

# Objective and gradient
f(x) = x' * Q * x
grad_f(x) = 2 * Q * x
x0 = rand(n); x0 ./= sum(x0)

# Linear Minimization Oracle for the simplex
function lmo(g)
    i = argmin(g)
    v = zeros(n)
    v[i] = 1.0
    return v
end

# Check if a point is new
function is_new_point(x, points, tol=TOL_UNIQUE)
    all(norm(x - p) >= tol for p in points)
end

# ================== Adaptive Frank-Wolfe Algorithm ================== #
function conditional_gradient_adjustable_scaling(f, grad_f, lmo, x0; max_iter=1000, epsilon=TOL_CONVERGENCE, delta=1e-10, beta=2, gamma=0.5)
    x_prev = copy(x0)
    x_curr = copy(x0)
    values = [f(x_curr)]
    times = [0.0]
    gaps = Float64[]
    L_ks = Float64[]
    steps = Float64[]
    backtrack_counts = Int[]
    gamma_history = [gamma]
    k = 0
    prev_grad = grad_f(x_prev)
    current_f = f(x_curr)
    recent_backtracks = Int[]

    while k < max_iter
        start = time()
        current_grad = grad_f(x_curr)
        v = lmo(current_grad)
        d = v - x_curr
        normd2 = dot(d, d)
        gap = -dot(current_grad, d)

        if gap <= epsilon
            push!(times, time() - start)
            push!(gaps, gap)
            break
        end

        if k == 0
            d0 = ones(length(x0)) / sqrt(length(x0))
            x_temp = x0 + 1e-3 * d0
            L_k = gamma * (norm(grad_f(x0) - grad_f(x_temp)) / (1e-3 * norm(d0)) + delta)
        else
            grad_diff = norm(current_grad - prev_grad)
            x_diff = norm(x_curr - x_prev)
            L_k = gamma * (grad_diff / x_diff + delta)
        end
        Lknormd2 = L_k * normd2
        t_k = min(gap / Lknormd2, 1.0)
        i = 0
        while true
            x_new = x_curr + t_k * d
            new_f = f(x_new)
            if current_f - new_f >= t_k * gap - (Lknormd2 / 2) * t_k^2
                x_prev = copy(x_curr)
                x_curr = x_new
                push!(backtrack_counts, i)
                push!(recent_backtracks, i)
                break
            else
                L_k *= beta
                Lknormd2 = L_k * normd2
                t_k = min(gap / Lknormd2, 1.0)
                i += 1
            end
        end

        if k % 10 == 0 && k > 0
            total_backtracks = sum(recent_backtracks)
            if total_backtracks == 0
                gamma = max(1e-4, gamma * 0.9)
            elseif total_backtracks > 10
                gamma = min(1.0, gamma * 1.1)
            end
            push!(gamma_history, gamma)
            recent_backtracks = Int[]
        elseif k % 10 == 0
            push!(gamma_history, gamma)
            recent_backtracks = Int[]
        else
            push!(recent_backtracks, i)
        end
        k += 1
        iteration_time = time() - start

        current_f = f(x_curr)
        prev_grad = current_grad
        push!(gaps, gap)
        push!(steps, t_k)
        push!(values, current_f)
        push!(times, iteration_time)
        push!(L_ks, L_k)

        if k % 100 == 0
            # println("k=$k, gap=$(round(gap, digits=4)), t_k=$(round(t_k, digits=4)), L_k=$(round(L_ks[end], digits=6)), Time=$(round(times[end], digits=4)), f=$(round(values[end], digits=6))")
        end
    end
    total_time = sum(times)
    return (x_curr, values, times, gaps, L_ks, backtrack_counts, steps, gamma_history, total_time)
end

# ================== Interior Point via Slack (Paper’s Method) ================== #
function find_interior_point(p_i, sites)
    model = Model(HiGHS.Optimizer)
    set_silent(model)
    @variable(model, x[1:n] >= 0)
    @variable(model, τ)
    @constraint(model, sum(x) == 1)
    for j in 1:length(sites)
        if j == findfirst(s -> s ≈ p_i, sites)
            continue
        end
        p_j = sites[j]
        a = p_j - p_i
        b_val = 0.5 * dot(p_j + p_i, p_j - p_i)
        @constraint(model, dot(a, x) <= b_val + τ)
    end
    @objective(model, Min, τ)
    optimize!(model)
    if termination_status(model) == OPTIMAL
        x_sol = value.(x)
        τ_sol = value(τ)
        @info "Interior point LP: τ = $(round(τ_sol, digits=8)), Status = $(τ_sol < -1e-8 ? "Interior" : "Boundary")"
        return τ_sol < -1e-8 ? (x_sol, "Interior") : (x_sol, "Boundary")
    else
        @warn "Interior point LP failed: Infeasible or unbounded"
        if length(sites) == 1
            @info "Returning barycenter as fallback"
            return ones(n) / n, "Interior"
        end
        return nothing, "Infeasible"
    end
end

# ================== Voronoi Partitioning ================== #
function create_voronoi_partitions(sites)
    base_A = vcat(
        ones(1, n),  # sum(x) ≤ 1
        -ones(1, n),  # sum(x) ≥ 1 → -sum(x) ≤ -1
        Matrix{Float64}(I, n, n)  # x ≥ 0
    )
    base_b = vcat(
        1.0,
        -1.0,
        zeros(n)
    )
    partitions = []
    K = length(sites)
    if K == 1
        push!(partitions, (base_A, base_b))
        return partitions
    end
    for i in 1:K
        A_i = copy(base_A)
        b_i = copy(base_b)
        p_i = sites[i]
        for j in 1:K
            i == j && continue
            p_j = sites[j]
            a = (p_j - p_i)'
            b_val = 0.5 * dot(p_j + p_i, p_j - p_i)
            A_i = vcat(A_i, a)
            b_i = vcat(b_i, b_val)
        end
        push!(partitions, (A_i, b_i))
    end
    return partitions
end

# ================== Main VDCG Algorithm ================== #
function voronoi_conditional_gradient(Q, x0)
    # Modified to return total time and number of points
    x_init, _, cg_times, _, _, _, _, _, init_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, x0)
    total_time = init_time
    sites = [x0, x_init]
    num_points = 1  # Count initial CG call
    archive = [(x_init, f(x_init))]
    best_so_far = f(x_init)
    @printf("Initial solution: f(x) = %.6f\n", f(x_init))
    @info "Distance between x0 and x_init: $(norm(x0 - x_init))"

    for level in 1:MAX_LEVELS
        start_time = time()
        @printf("→ VDCG Level %d: %d sites, Current best f(x) = %.6f\n", level, length(sites), best_so_far)
        partitions = create_voronoi_partitions(sites)
        new_sites = []
        for (i, (A_cell, b_cell)) in enumerate(partitions)
            x_inner, status = find_interior_point(sites[i], sites)
            if x_inner === nothing || status == "Infeasible"
                @info "Skipping cell $i: $status"
                continue
            end
            x_stat, _, cg_times, _, _, _, _, _, cg_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, x_inner)
            total_time += cg_time
            num_points += 1  # Count each CG call
            obj = f(x_stat)
            all_points = [s for (s, _) in archive]
            if is_new_point(x_stat, all_points) && is_new_point(x_stat, new_sites)
                push!(new_sites, x_stat)
                push!(archive, (x_stat, obj))
                @printf("  New point found: f(x) = %.6f\n", obj)
                best_so_far = min(best_so_far, obj)
            else
                min_dist = minimum(norm(x_stat - p) for p in all_points)
                @info "Cell $i: New point not added, obj = $(round(obj, digits=6)), min distance = $(round(min_dist, digits=6)), better = $(obj < best_so_far)"
            end
        end
        elapsed = time() - start_time
        total_time += elapsed
        if isempty(new_sites)
            @printf("→ No new points found. Terminating at level %d.\n", level)
            @printf("  🕒 Level %d runtime: %.4f sec | Best objective: %.6f\n", level, elapsed, best_so_far)
            break
        else
            append!(sites, new_sites)
            @printf("  ✅ Level %d completed in %.4f sec | Best objective so far: %.6f\n", level, elapsed, best_so_far)
        end
    end
    best_idx = argmin([obj for (_, obj) in archive])
    x_best, f_best = archive[best_idx]
    return x_best, f_best, archive, sites, total_time, num_points
end

# ================== Load Problem Data ================== #
function load_Q_matrix()
    mat_file = "Problem_900x900(0.75)_4.mat"
    url = "https://github.com/INFORMSJoC/2024.0577/raw/refs/heads/main/data/StQO%20problems/%23Problem_900x900(0.75)_4.mat"
    if !isfile(mat_file)
        @info "Downloading $mat_file..."
        Downloads.download(url, mat_file)
    end
    @info "Loading Q matrix..."
    mat_data = matread(mat_file)
    global Q = mat_data["Q"]
    @info "Q loaded: size $(size(Q)), cond = $(cond(Q))"
    L = 2 * maximum(abs.(eigvals(Q)))
    @info "Lipschitz constant (L): $L"
end

# ================== Run Experiment ================== #
function main()
    load_Q_matrix()
    # Run VDCG
    @printf("\n🚀 Starting VDCG...\n")
    @time x_vdcg, f_vdcg, archive, sites, vdcg_time, vdcg_points = voronoi_conditional_gradient(Q, x0)
    @printf("\n✅ VDCG Best Objective: %.6f\n", f_vdcg)
    @printf("✅ VDCG Total Time: %.4f sec, Total Points: %d\n", vdcg_time, vdcg_points)
    # Standard CG
    @printf("📊 Running Standard CG...\n")
    x_std, _, _, _, _, _, _, _, std_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, x0)
    f_std = f(x_std)
    @printf("📊 Standard CG Objective: %.6f\n", f_std)
    # Multi-start CG with time
    @printf("🔍 Running Multi-start CG (limited by VDCG time )...\n")
    f_multi = Inf
    total_multi_time = 0.0
    points_evaluated = 0
    seed = 23
    while total_multi_time < vdcg_time
        Random.seed!(seed)
        start = time()
        x, _, _, _, _, _, _, _, cg_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, rand(n) ./ sum(rand(n)))
        total_multi_time += time() - start
        points_evaluated += 1
        f_multi = min(f_multi, f(x))
        @printf("  Seed %d: f(x) = %.6f, Time so far: %.4f sec, Points: %d\n", seed, f(x), total_multi_time, points_evaluated)
        seed += 1
    end
    @printf("🔍 Multi-start CG Best: %.6f (Time: %.4f sec, Points: %d)\n", f_multi, total_multi_time, points_evaluated)
    # Summary Table
    @printf("\n📋 Summary of Results Problem_900x900(0.75)_4.mat\n")
    @printf("┌──────────────────────┬─────────────────┬──────────────┬─────────────────┐\n")
    @printf("│ %-20s │ %-15s │ %-12s │ %-15s │\n", "Method", "Objective Value", "Runtime (sec)", "Points Evaluated")
    @printf("├──────────────────────┼─────────────────┼──────────────┼─────────────────┤\n")
    @printf("│ %-20s │ %-15.6f │ %-12.4f │ %-15d │\n", "VDCG", f_vdcg, vdcg_time, vdcg_points)
    @printf("│ %-20s │ %-15.6f │ %-12.4f │ %-15d │\n", "Standard CG", f_std, std_time, 1)
    @printf("│ %-20s │ %-15.6f │ %-12.4f │ %-15d │\n", "Multi-start CG", f_multi, total_multi_time, points_evaluated)
    @printf("└──────────────────────┴─────────────────┴──────────────┴─────────────────┘\n")
end
main()

[ Info: Downloading Problem_900x900(0.75)_4.mat...
[ Info: Loading Q matrix...
[ Info: Q loaded: size (900, 900), cond = 4846.398044262524



🚀 Starting VDCG...
Initial solution: f(x) = -7.344412


[ Info: Lipschitz constant (L): 2825.983914699518


→ VDCG Level 1: 2 sites, Current best f(x) = -7.344412


[ Info: Distance between x0 and x_init: 0.3895833953310939
[ Info: Interior point LP: τ = -0.07752035, Status = Interior


...

  New point found: f(x) = -7.097489


[ Info: Interior point LP: τ = -0.11811288, Status = Interior


  New point found: f(x) = -6.600760
  ✅ Level 8 completed in 223.5550 sec | Best objective so far: -7.387044


[ Info: Interior point LP: τ = -0.12720286, Status = Interior


421.845860 seconds (44.72 M allocations: 355.957 GiB, 5.31% gc time, 0.25% compilation time)

✅ VDCG Best Objective: -7.387044
✅ VDCG Total Time: 637.3222 sec, Total Points: 392
📊 Running Standard CG...
📊 Standard CG Objective: -7.344412
🔍 Running Multi-start CG (limited by VDCG time )...
  Seed 23: f(x) = -7.344408, Time so far: 0.9026 sec, Points: 1
  Seed 24: f(x) = -6.872758, Time so far: 1.8972 sec, Points: 2
  Seed 25: f(x) = -6.862441, Time so far: 2.8805 sec, Points: 3
  Seed 26: f(x) = -6.984524, Time so far: 3.9377 sec, Points: 4
  Seed 27: f(x) = -6.872421, Time so far: 4.8952 sec, Points: 5
  Seed 28: f(x) = -7.309125, Time so far: 5.7722 sec, Points: 6
  Seed 29: f(x) = -7.134840, Time so far: 6.6845 sec, Points: 7
  Seed 30: f(x) = -6.811521, Time so far: 7.5484 sec, Points: 8
  Seed 31: f(x) = -6.872773, Time so far: 8.5108 sec, Points: 9
  Seed 32: f(x) = -6.966236, Time so far: 9.4152 sec, Points: 10
  Seed 33: f(x) = -6.983348, Time so far: 10.2711 sec, Points: 11
  S

In [9]:
using JuMP, HiGHS       # final Problem_900x900(0.75)_5.mat
using LinearAlgebra
using MAT
using Downloads
using Printf
using Random
using Statistics

# Set random seed for reproducibility
Random.seed!(23)

# ================== Configuration ================== #
const MAX_LEVELS = 8
const TOL_UNIQUE = 1e-3
const TOL_CONVERGENCE = 1e-6

# ================== Global Data ================== #
n = 900
Q = zeros(n, n)

# Objective and gradient
f(x) = x' * Q * x
grad_f(x) = 2 * Q * x
x0 = rand(n); x0 ./= sum(x0)

# Linear Minimization Oracle for the simplex
function lmo(g)
    i = argmin(g)
    v = zeros(n)
    v[i] = 1.0
    return v
end

# Check if a point is new
function is_new_point(x, points, tol=TOL_UNIQUE)
    all(norm(x - p) >= tol for p in points)
end

# ================== Adaptive Frank-Wolfe Algorithm ================== #
function conditional_gradient_adjustable_scaling(f, grad_f, lmo, x0; max_iter=1000, epsilon=TOL_CONVERGENCE, delta=1e-10, beta=2, gamma=0.5)
    x_prev = copy(x0)
    x_curr = copy(x0)
    values = [f(x_curr)]
    times = [0.0]
    gaps = Float64[]
    L_ks = Float64[]
    steps = Float64[]
    backtrack_counts = Int[]
    gamma_history = [gamma]
    k = 0
    prev_grad = grad_f(x_prev)
    current_f = f(x_curr)
    recent_backtracks = Int[]

    while k < max_iter
        start = time()
        current_grad = grad_f(x_curr)
        v = lmo(current_grad)
        d = v - x_curr
        normd2 = dot(d, d)
        gap = -dot(current_grad, d)

        if gap <= epsilon
            push!(times, time() - start)
            push!(gaps, gap)
            break
        end

        if k == 0
            d0 = ones(length(x0)) / sqrt(length(x0))
            x_temp = x0 + 1e-3 * d0
            L_k = gamma * (norm(grad_f(x0) - grad_f(x_temp)) / (1e-3 * norm(d0)) + delta)
        else
            grad_diff = norm(current_grad - prev_grad)
            x_diff = norm(x_curr - x_prev)
            L_k = gamma * (grad_diff / x_diff + delta)
        end
        Lknormd2 = L_k * normd2
        t_k = min(gap / Lknormd2, 1.0)
        i = 0
        while true
            x_new = x_curr + t_k * d
            new_f = f(x_new)
            if current_f - new_f >= t_k * gap - (Lknormd2 / 2) * t_k^2
                x_prev = copy(x_curr)
                x_curr = x_new
                push!(backtrack_counts, i)
                push!(recent_backtracks, i)
                break
            else
                L_k *= beta
                Lknormd2 = L_k * normd2
                t_k = min(gap / Lknormd2, 1.0)
                i += 1
            end
        end

        if k % 10 == 0 && k > 0
            total_backtracks = sum(recent_backtracks)
            if total_backtracks == 0
                gamma = max(1e-4, gamma * 0.9)
            elseif total_backtracks > 10
                gamma = min(1.0, gamma * 1.1)
            end
            push!(gamma_history, gamma)
            recent_backtracks = Int[]
        elseif k % 10 == 0
            push!(gamma_history, gamma)
            recent_backtracks = Int[]
        else
            push!(recent_backtracks, i)
        end
        k += 1
        iteration_time = time() - start

        current_f = f(x_curr)
        prev_grad = current_grad
        push!(gaps, gap)
        push!(steps, t_k)
        push!(values, current_f)
        push!(times, iteration_time)
        push!(L_ks, L_k)

        if k % 100 == 0
            # println("k=$k, gap=$(round(gap, digits=4)), t_k=$(round(t_k, digits=4)), L_k=$(round(L_ks[end], digits=6)), Time=$(round(times[end], digits=4)), f=$(round(values[end], digits=6))")
        end
    end
    total_time = sum(times)
    return (x_curr, values, times, gaps, L_ks, backtrack_counts, steps, gamma_history, total_time)
end

# ================== Interior Point via Slack (Paper’s Method) ================== #
function find_interior_point(p_i, sites)
    model = Model(HiGHS.Optimizer)
    set_silent(model)
    @variable(model, x[1:n] >= 0)
    @variable(model, τ)
    @constraint(model, sum(x) == 1)
    for j in 1:length(sites)
        if j == findfirst(s -> s ≈ p_i, sites)
            continue
        end
        p_j = sites[j]
        a = p_j - p_i
        b_val = 0.5 * dot(p_j + p_i, p_j - p_i)
        @constraint(model, dot(a, x) <= b_val + τ)
    end
    @objective(model, Min, τ)
    optimize!(model)
    if termination_status(model) == OPTIMAL
        x_sol = value.(x)
        τ_sol = value(τ)
        @info "Interior point LP: τ = $(round(τ_sol, digits=8)), Status = $(τ_sol < -1e-8 ? "Interior" : "Boundary")"
        return τ_sol < -1e-8 ? (x_sol, "Interior") : (x_sol, "Boundary")
    else
        @warn "Interior point LP failed: Infeasible or unbounded"
        if length(sites) == 1
            @info "Returning barycenter as fallback"
            return ones(n) / n, "Interior"
        end
        return nothing, "Infeasible"
    end
end

# ================== Voronoi Partitioning ================== #
function create_voronoi_partitions(sites)
    base_A = vcat(
        ones(1, n),  # sum(x) ≤ 1
        -ones(1, n),  # sum(x) ≥ 1 → -sum(x) ≤ -1
        Matrix{Float64}(I, n, n)  # x ≥ 0
    )
    base_b = vcat(
        1.0,
        -1.0,
        zeros(n)
    )
    partitions = []
    K = length(sites)
    if K == 1
        push!(partitions, (base_A, base_b))
        return partitions
    end
    for i in 1:K
        A_i = copy(base_A)
        b_i = copy(base_b)
        p_i = sites[i]
        for j in 1:K
            i == j && continue
            p_j = sites[j]
            a = (p_j - p_i)'
            b_val = 0.5 * dot(p_j + p_i, p_j - p_i)
            A_i = vcat(A_i, a)
            b_i = vcat(b_i, b_val)
        end
        push!(partitions, (A_i, b_i))
    end
    return partitions
end

# ================== Main VDCG Algorithm ================== #
function voronoi_conditional_gradient(Q, x0)
    # Modified to return total time and number of points
    x_init, _, cg_times, _, _, _, _, _, init_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, x0)
    total_time = init_time
    sites = [x0, x_init]
    num_points = 1  # Count initial CG call
    archive = [(x_init, f(x_init))]
    best_so_far = f(x_init)
    @printf("Initial solution: f(x) = %.6f\n", f(x_init))
    @info "Distance between x0 and x_init: $(norm(x0 - x_init))"

    for level in 1:MAX_LEVELS
        start_time = time()
        @printf("→ VDCG Level %d: %d sites, Current best f(x) = %.6f\n", level, length(sites), best_so_far)
        partitions = create_voronoi_partitions(sites)
        new_sites = []
        for (i, (A_cell, b_cell)) in enumerate(partitions)
            x_inner, status = find_interior_point(sites[i], sites)
            if x_inner === nothing || status == "Infeasible"
                @info "Skipping cell $i: $status"
                continue
            end
            x_stat, _, cg_times, _, _, _, _, _, cg_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, x_inner)
            total_time += cg_time
            num_points += 1  # Count each CG call
            obj = f(x_stat)
            all_points = [s for (s, _) in archive]
            if is_new_point(x_stat, all_points) && is_new_point(x_stat, new_sites)
                push!(new_sites, x_stat)
                push!(archive, (x_stat, obj))
                @printf("  New point found: f(x) = %.6f\n", obj)
                best_so_far = min(best_so_far, obj)
            else
                min_dist = minimum(norm(x_stat - p) for p in all_points)
                @info "Cell $i: New point not added, obj = $(round(obj, digits=6)), min distance = $(round(min_dist, digits=6)), better = $(obj < best_so_far)"
            end
        end
        elapsed = time() - start_time
        total_time += elapsed
        if isempty(new_sites)
            @printf("→ No new points found. Terminating at level %d.\n", level)
            @printf("  🕒 Level %d runtime: %.4f sec | Best objective: %.6f\n", level, elapsed, best_so_far)
            break
        else
            append!(sites, new_sites)
            @printf("  ✅ Level %d completed in %.4f sec | Best objective so far: %.6f\n", level, elapsed, best_so_far)
        end
    end
    best_idx = argmin([obj for (_, obj) in archive])
    x_best, f_best = archive[best_idx]
    return x_best, f_best, archive, sites, total_time, num_points
end

# ================== Load Problem Data ================== #
function load_Q_matrix()
    mat_file = "Problem_900x900(0.75)_5.mat"
    url = "https://github.com/INFORMSJoC/2024.0577/raw/refs/heads/main/data/StQO%20problems/%23Problem_900x900(0.75)_5.mat"
    if !isfile(mat_file)
        @info "Downloading $mat_file..."
        Downloads.download(url, mat_file)
    end
    @info "Loading Q matrix..."
    mat_data = matread(mat_file)
    global Q = mat_data["Q"]
    @info "Q loaded: size $(size(Q)), cond = $(cond(Q))"
    L = 2 * maximum(abs.(eigvals(Q)))
    @info "Lipschitz constant (L): $L"
end

# ================== Run Experiment ================== #
# ================== Run Experiment ================== #
function main()
    load_Q_matrix()
    # Run VDCG
    @printf("\n🚀 Starting VDCG...\n")
    @time x_vdcg, f_vdcg, archive, sites, vdcg_time, vdcg_points = voronoi_conditional_gradient(Q, x0)
    @printf("\n✅ VDCG Best Objective: %.6f\n", f_vdcg)
    @printf("✅ VDCG Total Time: %.4f sec, Total Points: %d\n", vdcg_time, vdcg_points)
    # Standard CG
    @printf("📊 Running Standard CG...\n")
    x_std, _, _, _, _, _, _, _, std_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, x0)
    f_std = f(x_std)
    @printf("📊 Standard CG Objective: %.6f\n", f_std)
    # Multi-start CG with time
    @printf("🔍 Running Multi-start CG (limited by VDCG time )...\n")
    f_multi = Inf
    total_multi_time = 0.0
    points_evaluated = 0
    seed = 23
    while total_multi_time < vdcg_time
        Random.seed!(seed)
        start = time()
        x, _, _, _, _, _, _, _, cg_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, rand(n) ./ sum(rand(n)))
        total_multi_time += time() - start
        points_evaluated += 1
        f_multi = min(f_multi, f(x))
        @printf("  Seed %d: f(x) = %.6f, Time so far: %.4f sec, Points: %d\n", seed, f(x), total_multi_time, points_evaluated)
        seed += 1
    end
    @printf("🔍 Multi-start CG Best: %.6f (Time: %.4f sec, Points: %d)\n", f_multi, total_multi_time, points_evaluated)
    # Summary Table
    @printf("\n📋 Summary of Results Problem_900x900(0.75)_5.mat\n")
    @printf("┌──────────────────────┬─────────────────┬──────────────┬─────────────────┐\n")
    @printf("│ %-20s │ %-15s │ %-12s │ %-15s │\n", "Method", "Objective Value", "Runtime (sec)", "Points Evaluated")
    @printf("├──────────────────────┼─────────────────┼──────────────┼─────────────────┤\n")
    @printf("│ %-20s │ %-15.6f │ %-12.4f │ %-15d │\n", "VDCG", f_vdcg, vdcg_time, vdcg_points)
    @printf("│ %-20s │ %-15.6f │ %-12.4f │ %-15d │\n", "Standard CG", f_std, std_time, 1)
    @printf("│ %-20s │ %-15.6f │ %-12.4f │ %-15d │\n", "Multi-start CG", f_multi, total_multi_time, points_evaluated)
    @printf("└──────────────────────┴─────────────────┴──────────────┴─────────────────┘\n")
end
main()

[ Info: Downloading Problem_900x900(0.75)_5.mat...
[ Info: Loading Q matrix...
[ Info: Q loaded: size (900, 900), cond = 40075.37419608342



🚀 Starting VDCG...
Initial solution: f(x) = -6.871050


[ Info: Lipschitz constant (L): 2880.1646223209054


→ VDCG Level 1: 2 sites, Current best f(x) = -6.871050


[ Info: Distance between x0 and x_init: 0.44360012568499935
[ Info: Interior point LP: τ = -0.10010239, Status = Interior


...

  New point found: f(x) = -6.897638


[ Info: Interior point LP: τ = -0.0746401, Status = Interior


  New point found: f(x) = -6.833421
  ✅ Level 8 completed in 259.6269 sec | Best objective so far: -7.392474


[ Info: Interior point LP: τ = -0.12826664, Status = Interior


480.365178 seconds (52.87 M allocations: 443.662 GiB, 4.98% gc time, 0.18% compilation time)

✅ VDCG Best Objective: -7.392474
✅ VDCG Total Time: 723.0110 sec, Total Points: 425
📊 Running Standard CG...
📊 Standard CG Objective: -6.871050
🔍 Running Multi-start CG (limited by VDCG time )...
  Seed 23: f(x) = -6.911930, Time so far: 0.7837 sec, Points: 1
  Seed 24: f(x) = -6.968881, Time so far: 1.7123 sec, Points: 2
  Seed 25: f(x) = -6.809019, Time so far: 2.6026 sec, Points: 3
  Seed 26: f(x) = -6.871019, Time so far: 3.4917 sec, Points: 4
  Seed 27: f(x) = -6.660728, Time so far: 4.2888 sec, Points: 5
  Seed 28: f(x) = -6.991743, Time so far: 4.9884 sec, Points: 6
  Seed 29: f(x) = -6.968941, Time so far: 5.7930 sec, Points: 7
  Seed 30: f(x) = -6.969705, Time so far: 6.5540 sec, Points: 8
  Seed 31: f(x) = -6.660661, Time so far: 7.3307 sec, Points: 9
  Seed 32: f(x) = -6.991788, Time so far: 8.1153 sec, Points: 10
  Seed 33: f(x) = -7.185775, Time so far: 8.8844 sec, Points: 11
  Se